In [1]:
import os
import pandas as pd
import glob

def count_fire_years(fire_folder):
    # Find all annual CSV files
    fire_files = sorted(glob.glob(os.path.join(fire_folder, "FireCCI51_BurnedPixels_Catchments_Weekly_*.csv")))

    lake_fire_years = {}

    for file in fire_files:
        try:
            year_df = pd.read_csv(file)
            fire_cols = year_df.columns[5:]  # after week_end

            yearly_sum = year_df.groupby("Lake_ID")[fire_cols].sum()
            fire_total = yearly_sum.sum(axis=1)

            for lake_id, total in fire_total.items():
                if lake_id not in lake_fire_years:
                    lake_fire_years[lake_id] = 0
                if total > 0:
                    lake_fire_years[lake_id] += 1
        except Exception as e:
            print(f"Error reading {file}: {e}")

    # Convert to DataFrame
    fire_experience_df = pd.DataFrame([
        {"Lake_ID": lake_id, "Years_with_Fire": count}
        for lake_id, count in lake_fire_years.items()
    ])

    return fire_experience_df




In [2]:
# 🔁 Example usage:
df = count_fire_years("Datasets/GEE/firecci_raw")
df.to_csv("Datasets/GEE/lake_fire_experience_2002_2019.csv", index=False)

In [3]:
print(f"{(df['Years_with_Fire'] > 0).sum()} out of {len(df)} lakes experienced fire.")


90 out of 466 lakes experienced fire.
